<!--BOOK_INFORMATION-->
<img align="left" style="padding-right:10px;" src="figures/PDSH-cover-small.png">
*This notebook contains an excerpt from the [Python Data Science Handbook](http://shop.oreilly.com/product/0636920034919.do) by Jake VanderPlas; the content is available [on GitHub](https://github.com/jakevdp/PythonDataScienceHandbook).*

*The text is released under the [CC-BY-NC-ND license](https://creativecommons.org/licenses/by-nc-nd/3.0/us/legalcode), and code is released under the [MIT license](https://opensource.org/licenses/MIT). If you find this content useful, please consider supporting the work by [buying the book](http://shop.oreilly.com/product/0636920034919.do)!*

<!--NAVIGATION-->
< [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb) | [Contents](Index.ipynb) | [Handling Missing Data](03.04-Missing-Values.ipynb) >

# Operating on Data in Pandas

One of the essential pieces of **NumPy is the ability to perform quick element-wise operations**, both with basic arithmetic (addition, subtraction, multiplication, etc.) and with more sophisticated operations (trigonometric functions, exponential and logarithmic functions, etc.).
**Pandas inherits much of this functionality from NumPy, and the ufuncs** that we introduced in [Computation on NumPy Arrays: Universal Functions](02.03-Computation-on-arrays-ufuncs.ipynb) are key to this.

Pandas includes a couple useful twists, however: **for unary operations like negation and trigonometric functions, these ufuncs will *preserve index and column labels* in the output**, and for binary operations such as addition and multiplication, Pandas will automatically <u>*align indices* when passing the objects to the ufunc.</u>
This means that keeping the context of data and combining data from different sources–both potentially error-prone tasks with raw NumPy arrays–become essentially foolproof ones with Pandas.
We will additionally see that there are well-defined operations between one-dimensional ``Series`` structures and two-dimensional ``DataFrame`` structures.

 numpy의 essential pieces: element-wise operations > universal functions > preserve index and column labels  > align index 

## Ufuncs: Index Preservation

Because Pandas is designed to work with NumPy, any NumPy ufunc will work on Pandas ``Series`` and ``DataFrame`` objects.
Let's start by defining a simple ``Series`` and ``DataFrame`` on which to demonstrate this:

In [1]:
import pandas as pd
import numpy as np

In [2]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0, 10, 4))
ser

0    6
1    3
2    7
3    4
dtype: int32

In [3]:
print(ser)

0    6
1    3
2    7
3    4
dtype: int32


In [4]:
type(ser)

pandas.core.series.Series

In [5]:
a = rng.randint(1, 12, (3,4))
a

array([[ 7, 10,  3,  7],
       [11, 11,  8,  5],
       [ 4,  8,  8,  3]], dtype=int32)

In [ ]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)), #0에서 10사이의 난수를 (3,4)배열로
                  columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4


In [8]:
print(df.loc[0:1,:])

   A  B  C  D
0  5  4  1  7
1  5  1  4  0


If we apply a<u> NumPy ufunc </u>on either of these objects, the result will be another Pandas object <u>*with the indices preserved:*</u>

In [ ]:
np.exp(ser) # e (x)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

Or, for a slightly more complex calculation:

In [ ]:
np.sin(df * np.pi / 4) # preserve index

,A,B,C,D
0,-0.707107,1.224647e-16,7.071068e-01,-0.707107
1,-0.707107,7.071068e-01,1.224647e-16,0.000000
2,0.707107,-7.071068e-01,-2.449294e-16,0.000000


Any of the ufuncs discussed in [Computation on NumPy Arrays: Universal Functions](02.03-Computation-on-arrays-ufuncs.ipynb) can be used in a similar manner.

## UFuncs: Index Alignment

For binary operations on two ``Series`` or ``DataFrame`` objects, Pandas will **align indices in the process of performing the operation.**
This is very convenient when working with incomplete data, as we'll see in some of the examples that follow.

### Index alignment in Series

As an example, suppose we are **combining two different data sources**, and find only the top three US states by *area* and the top three US states by *population*:

In [8]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662,
                  'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193,
                        'New York': 19651127}, name='population')

Let's see what happens when we divide these to compute the population density:

In [11]:
area

Alaska        1723337
Texas          695662
California     423967
Name: area, dtype: int64

In [4]:
population / area

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

The resulting array contains the *union* of indices of the two input arrays, which could be determined using standard Python set arithmetic on these indices:

In [5]:
area.index

Index(['Alaska', 'Texas', 'California'], dtype='object')

In [6]:
population.index

Index(['California', 'Texas', 'New York'], dtype='object')

In [5]:
# area.index | population.index #에러 발생 - 교재 버젼임 
result = area.index.union(population.index)
print(result)

NameError: name 'area' is not defined

Any item for which one or the other does not have an entry is **marked with ``NaN``, or "Not a Number," which is how Pandas marks missing data **(see further discussion of missing data in [Handling Missing Data](03.04-Missing-Values.ipynb)).
This index matching is implemented this way for any of Python's built-in arithmetic expressions; any missing values are filled in with NaN by default:

In [4]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A + B # 인덱스를 맞추고 연산 수행 

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

If using NaN values is not the desired behavior, the fill value can be modified using appropriate object methods in place of the operators.
For example, calling ``A.add(B)`` is equivalent to calling ``A + B``, but allows optional explicit specification of the fill value for any elements in ``A`` or ``B`` that might be missing:

In [8]:
A.add(B, fill_value=0)

NotImplementedError: fill_value 0 not supported.

### Index alignment in DataFrame

A similar type of alignment takes place for *both* columns and indices when performing operations on ``DataFrame``s:

In [26]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)),
                 columns=list('AB'))
A

,A,B
0,7,10
1,18,16


In [14]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)),
                 columns=list('BAC'))
B

,B,A,C
0,1,9,8
1,9,4,1
2,3,6,7


In [ ]:
A + B # 인덱스로 align

,A,B,C
0,10.0,25.0,NaN
1,17.0,17.0,NaN
2,NaN,NaN,NaN


In [16]:
A

,A,B
0,6,17
1,3,13


In [33]:
B

,B,A,C
0,8,1,9
1,8,9,4
2,1,3,6


Notice that <u> indices are aligned correctly irrespective of their order in the two objects </u>, and indices in the result are sorted.
As was the case with ``Series``, we can use the associated object's arithmetic method and pass any desired ``fill_value`` to be used in place of missing entries.
Here we'll fill with the mean of all values in ``A`` (computed by first stacking the rows of ``A``):

In [15]:
As = A.stack()
print(As)

0  A     6
   B    17
1  A     3
   B    13
dtype: int32


In [ ]:
fill = A.stack().mean() # DataFrame A의 모든 요소를 일렬로 펼쳐서 평균을 구한다.
A.add(B, fill_value=fill) 

,A,B,C
0,10.0,25.0,14.0
1,17.0,17.0,13.0
2,15.0,16.0,12.0


In [35]:
fill

np.float64(8.5)

The following table lists Python operators and their equivalent Pandas object methods:

| Python Operator | Pandas Method(s)                      |
|-----------------|---------------------------------------|
| ``+``           | ``add()``                             |
| ``-``           | ``sub()``, ``subtract()``             |
| ``*``           | ``mul()``, ``multiply()``             |
| ``/``           | ``truediv()``, ``div()``, ``divide()``|
| ``//``          | ``floordiv()``                        |
| ``%``           | ``mod()``                             |
| ``**``          | ``pow()``                             |


## Ufuncs: Operations Between DataFrame and Series

When performing **operations between a ``DataFrame`` and a ``Series``, the index and column alignment is similarly maintained.**
Operations between a ``DataFrame`` and a ``Series`` are similar to operations between a two-dimensional and one-dimensional NumPy array.
Consider one common operation, where we find the difference of a two-dimensional array and one of its rows:

In [27]:
A = rng.randint(10, size=(3, 4))
A

array([[7, 2, 2, 0],
       [4, 9, 6, 9],
       [8, 6, 8, 7]], dtype=int32)

In [28]:
A - A[0] # (4,) > (1,4) > (3,4)

array([[ 0,  0,  0,  0],
       [-3,  7,  4,  9],
       [ 1,  4,  6,  7]], dtype=int32)

According to <u>NumPy's broadcasting rules </u>(see [Computation on Arrays: Broadcasting](02.05-Computation-on-arrays-broadcasting.ipynb)), subtraction between a two-dimensional array and one of its rows is applied row-wise.

In Pandas, the convention similarly operates row-wise by default:

In [ ]:
df = pd.DataFrame(A, columns=list('QRST'))
df - df.iloc[0] # broadcasting rule

,Q,R,S,T
0,0,0,0,0
1,-7,4,4,4
2,-7,4,1,2


If you would instead like to operate column-wise, you can use the object methods mentioned earlier, while specifying the ``axis`` keyword:

In [42]:
df['R']

0    2
1    7
2    5
Name: R, dtype: int32

In [41]:
df

,Q,R,S,T
0,7,2,0,3
1,1,7,3,1
2,5,5,9,3


In [ ]:
df.subtract(df['R'], axis=0) # (3,1) -> (3,4)

,Q,R,S,T
0,5,0,-2,1
1,-6,0,-4,-6
2,0,0,4,-2


Note that these ``DataFrame``/``Series`` operations, like the operations discussed above, will automatically align  indices between the two elements:

In [44]:
halfrow = df.iloc[0, ::2]
halfrow

Q    7
S    0
Name: 0, dtype: int32

In [45]:
df - halfrow

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,-6.0,NaN,3.0,NaN
2,-2.0,NaN,9.0,NaN


This <u>preservation and alignment of indices and columns </u> means that operations on data in Pandas will always maintain the data context, which prevents the types of silly errors that might come up when working with heterogeneous and/or misaligned data in raw NumPy arrays.

<!--NAVIGATION-->
< [Data Indexing and Selection](03.02-Data-Indexing-and-Selection.ipynb) | [Contents](Index.ipynb) | [Handling Missing Data](03.04-Missing-Values.ipynb) >